In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import time

In [2]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding="SAME")


def max_pool_2x2(x):
    return tf.nn.max_pool(
        x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME"
    )


def conv_layer(input, shape):
    W = weight_variable(shape)
    b = bias_variable([shape[3]])
    return tf.nn.relu(conv2d(input, W) + b)


def full_layer(input, size):
    in_size = int(input.get_shape()[1])
    W = weight_variable([in_size, size])
    b = bias_variable([size])
    return tf.matmul(input, W) + b

In [5]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])
keep_prob = tf.placeholder(tf.float32)
x_image = tf.reshape(x, [-1, 28, 28, 1])

with tf.name_scope("conv_1"):
    conv1 = conv_layer(x_image, shape=[5, 5, 1, 32])
    conv1_pool = max_pool_2x2(conv1)

with tf.name_scope("conv_2"):
    conv2 = conv_layer(conv1_pool, shape=[5, 5, 32, 64])
    conv2_pool = max_pool_2x2(conv2)
    conv2_flat = tf.reshape(conv2_pool, [-1, 7 * 7 * 64])

with tf.name_scope("full_1"):
    full_1 = tf.nn.relu(full_layer(conv2_flat, 1024))

with tf.name_scope("dropout"):
    full1_drop = tf.nn.dropout(full_1, keep_prob=keep_prob)

with tf.name_scope("activations"):
    y_conv = full_layer(full1_drop, 10)
    tf.summary.scalar("cross_entropy_loss", y_conv )
    mnist = input_data.read_data_sets("tmp/data", one_hot=True )

with tf.name_scope("cross"):
    cross_entropy = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(
            logits=y_conv, labels=y_
        )
    )  

INFO:tensorflow:Summary name  cross_entropy_loss is illegal; using _cross_entropy_loss instead.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting tmp/data/t10k-labels-idx1-ubyte.gz


In [8]:
# SGD
train_step = tf.train.AdagradOptimizer(learning_rate=1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    start_time = time.time()

    for i in range(50000):
        batch = mnist.train.next_batch(50)
        if i % 100 == 0:
            train_accuracy = sess.run(
                accuracy, feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0}
            )
            print("time {} , step {} , training accuracy {}".format(
                time.time() - start_time, i, train_accuracy
            ))
        sess.run(train_step, feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

    X = mnist.test.images.reshape(10, 1000, 784)
    Y = mnist.test.labels.reshape(10, 1000, 10)
    test_accuracy = np.mean(
        [
            sess.run(accuracy, feed_dict={x: X[i], y_: Y[i], keep_prob: 1.0})
            for i in range(10)
        ]
    )
    print(" test accuracy : {}".format(test_accuracy))


time 0.038308143615722656 , step 0 , training accuracy 0.07999999821186066
time 4.743762969970703 , step 100 , training accuracy 0.18000000715255737
time 9.155816078186035 , step 200 , training accuracy 0.2199999988079071
time 13.745874166488647 , step 300 , training accuracy 0.2800000011920929
time 18.56551504135132 , step 400 , training accuracy 0.3799999952316284
time 23.645045042037964 , step 500 , training accuracy 0.5
time 28.550722122192383 , step 600 , training accuracy 0.5
time 33.06458401679993 , step 700 , training accuracy 0.6000000238418579
time 37.533385038375854 , step 800 , training accuracy 0.5
time 42.13263010978699 , step 900 , training accuracy 0.6000000238418579
time 46.84612512588501 , step 1000 , training accuracy 0.6200000047683716
time 51.73351311683655 , step 1100 , training accuracy 0.699999988079071
time 56.59836220741272 , step 1200 , training accuracy 0.5799999833106995
time 61.7056040763855 , step 1300 , training accuracy 0.6800000071525574
time 66.645384

KeyboardInterrupt: 

In [9]:
from tensorflow . python . keras . datasets import cifar10

In [10]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 29s 0us/step
